In [1]:
import pandas as pd
from sklearn import metrics
import glob
from sklearn.model_selection import train_test_split

In [2]:
path = '../Dataset/CICIoT2023/'
# data = pd.read_csv(path+'part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv')
data = pd.concat([pd.read_csv(f) for f in glob.glob(path+'*.csv')])

In [5]:
data.head(5)

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,54.00,6.00,64.00,0.329807,0.329807,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334383e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
1,0.000000,57.04,6.33,64.00,4.290556,4.290556,0.0,0.0,0.0,0.0,...,2.822973,57.04,8.292607e+07,9.5,10.464666,4.010353,160.987842,0.05,141.55,DoS-TCP_Flood
2,0.000000,0.00,1.00,64.00,33.396799,33.396799,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.312799e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,DDoS-ICMP_Flood
3,0.328175,76175.00,17.00,64.00,4642.133010,4642.133010,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.301570e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,DoS-UDP_Flood
4,0.117320,101.73,6.11,65.91,6.202211,6.202211,0.0,0.0,1.0,0.0,...,23.113111,57.88,8.297300e+07,9.5,11.346876,32.716243,3016.808286,0.19,141.55,DoS-SYN_Flood


In [6]:
category_dict = {
        'DDoS-ACK_Fragmentation' : 1,
        'DDoS-HTTP_Flood' : 1,
        'DDoS-ICMP_Flood': 1,
        'DDoS-PSHACK_Flood': 1,
        'DDoS-RSTFINFlood': 1,
        'DDoS-SYN_Flood': 1,
        'DDoS-SlowLoris': 1,
        'DDoS-SynonymousIP_Flood': 1,
        'DDoS-TCP_Flood': 1,
        'DDoS-UDP_Flood': 1,
        'DDoS-UDP_Fragmentation': 1,
        'DDoS-ICMP_Fragmentation' : 1,

        'DoS-HTTP_Flood' : 1,
        'DoS-SYN_Flood' : 1,
        'DoS-TCP_Flood' : 1,
        'DoS-UDP_Flood' : 1,

        'DictionaryBruteForce' : 1,

        'MITM-ArpSpoofing' : 1,
        'DNS_Spoofing' : 1,

        'Recon-HostDiscovery' : 1,
        'Recon-OSScan' : 1,
        'Recon-PingSweep' : 1,
        'Recon-PortScan' : 1,
        'VulnerabilityScan' : 1,

        'SqlInjection' : 1,
        'CommandInjection' : 1,
        'Backdoor_Malware' : 1,
        'Uploading_Attack' : 1,
        'XSS' : 1,
        'BrowserHijacking' : 1,

        'Mirai-greeth_flood' : 1,
        'Mirai-greip_flood' : 1,
        'Mirai-udpplain' : 1,

        'BenignTraffic' : 0
    }

In [7]:
data['label'] = data['label'].map(category_dict)

In [4]:
import xgboost
xgb_model = xgboost.XGBClassifier(objective='binary:logistic', verbosity=2, n_estimators=100, max_depth=5)

In [8]:
data.label.value_counts()

label
1    45588384
0     1098195
Name: count, dtype: int64

In [9]:
X, y = data[data.columns[:-1]], data.label

In [10]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [11]:
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [12]:
st_cross_val_results = cross_val_score(xgb_model, X, y, cv=skf)
print(f'Stratified Cross-Validation Results (Accuracy): {st_cross_val_results}')
print(f'Mean Accuracy: {st_cross_val_results.mean()}')

# Stratified Cross-Validation Results (Accuracy): [0.9966862  0.99668941 0.99668181 0.9967104  0.99670001]
# Mean Accuracy: 0.9966935679739233

Stratified Cross-Validation Results (Accuracy): [0.9966862  0.99668941 0.99668181 0.9967104  0.99670001]
Mean Accuracy: 0.9966935679739233


In [6]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [7]:
train.to_csv('../Dataset/CIC_IOT_2023_20%_training_2',index=None)

In [8]:
test.to_csv('../Dataset/CIC_IOT_2023_20%_testing_2.csv',index=None)